# Detecting Malicious URLs using Machine Learning

Import appropriate packages.

In [49]:
# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Measuring Accuracy
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_roc_curve

# Data processing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Save and load models
import pickle

# Managing Datasets
import pandas as pd
# Extra Graphs
import seaborn as sns
# Maths
import numpy as np

Scale the data to accomodate to some ML models such as KNN.

In [50]:
ml_data = pd.read_csv("ml_data/ml_data.csv")

In [51]:
ml_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 142 entries, Unnamed: 0 to x_powered_by_science!
dtypes: float64(9), int64(118), object(15)
memory usage: 111.1+ KB


In [52]:
ml_data.head()

,Unnamed: 0,Unnamed: 0.1,label,url_type,scheme,netloc,path,params,query,fragment,...,asn_reg_arin,asn_reg_lacnic,asn_reg_ripencc,x_powered_by_ASP.NET,x_powered_by_Express,x_powered_by_PHP/5.4.16,x_powered_by_PHP/5.6.40,x_powered_by_PHP/7.1.33,x_powered_by_W3 Total Cache/0.11.0,x_powered_by_science!
0,179,68538,0,1,http,www.development.ohio.gov,/research/files/s0.htm,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
1,240,977286,1,0,http,167.71.68.6,/sh,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
2,148,47396,0,1,https,www.answers.yahoo.com,/question/index,NaN,qid=20111110162954AAaTxV8,NaN,...,0,0,1,0,0,0,0,0,0,0
3,352,227572,1,0,http,119.180.115.146:56436,/Mozi.m,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,361,925170,1,1,http,onedrive.linkpc.net,/Ali/Yasine/final.txt,NaN,NaN,NaN,...,1,0,0,0,0,1,0,0,0,0


In [53]:
data=ml_data
data = data.drop('scheme', axis=1)
data = data.drop('netloc', axis=1)
data = data.drop('path', axis=1)
data = data.drop('params', axis=1)
data = data.drop('query', axis=1)
data = data.drop('fragment', axis=1)
data = data.drop('ip', axis=1)
data = data.drop('top_level_domain', axis=1)
data = data.drop('asn_reg', axis=1)
data = data.drop('asn_country_code', axis=1)
data = data.drop('asn_date', axis=1)
data = data.drop('nets_country_code', axis=1)
data = data.drop('is_redirect', axis=1)
data = data.drop('server', axis=1)
data = data.drop('expires', axis=1)
data = data.drop('content_type', axis=1)
data = data.drop('x_powered_by', axis=1)
data

,Unnamed: 0,Unnamed: 0.1,label,url_type,port,ip_1,ip_2,ip_3,ip_4,contains [,...,asn_reg_arin,asn_reg_lacnic,asn_reg_ripencc,x_powered_by_ASP.NET,x_powered_by_Express,x_powered_by_PHP/5.4.16,x_powered_by_PHP/5.6.40,x_powered_by_PHP/7.1.33,x_powered_by_W3 Total Cache/0.11.0,x_powered_by_science!
0,179,68538,0,1,80,66.0,144.0,126.0,8.0,0,...,1,0,0,0,0,0,0,0,0,0
1,240,977286,1,0,80,167.0,71.0,68.0,6.0,0,...,1,0,0,0,0,0,0,0,0,0
2,148,47396,0,1,443,87.0,248.0,100.0,201.0,0,...,0,0,1,0,0,0,0,0,0,0
3,352,227572,1,0,56436,119.0,180.0,115.0,146.0,0,...,0,0,0,0,0,0,0,0,0,0
4,361,925170,1,1,80,67.0,214.0,175.0,69.0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,357,870863,1,1,80,108.0,167.0,189.0,73.0,0,...,1,0,0,0,0,0,0,0,0,0
96,161,66423,0,1,80,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
97,355,1018578,1,0,80,209.0,97.0,178.0,68.0,0,...,1,0,0,0,0,0,0,0,0,0
98,21,280102,0,1,443,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
data = data.dropna()
data

,Unnamed: 0,Unnamed: 0.1,label,url_type,port,ip_1,ip_2,ip_3,ip_4,contains [,...,asn_reg_arin,asn_reg_lacnic,asn_reg_ripencc,x_powered_by_ASP.NET,x_powered_by_Express,x_powered_by_PHP/5.4.16,x_powered_by_PHP/5.6.40,x_powered_by_PHP/7.1.33,x_powered_by_W3 Total Cache/0.11.0,x_powered_by_science!
1,240,977286,1,0,80,167.0,71.0,68.0,6.0,0,...,1,0,0,0,0,0,0,0,0,0
2,148,47396,0,1,443,87.0,248.0,100.0,201.0,0,...,0,0,1,0,0,0,0,0,0,0
4,361,925170,1,1,80,67.0,214.0,175.0,69.0,0,...,1,0,0,0,0,1,0,0,0,0
5,272,67775,1,1,80,183.0,181.0,89.0,149.0,0,...,0,0,0,0,0,0,0,0,0,0
7,128,256160,0,1,443,2.0,19.0,148.0,208.0,0,...,0,0,1,0,0,0,0,0,0,0
10,178,233607,0,1,443,157.0,240.0,1.0,35.0,0,...,1,0,0,0,0,0,0,0,0,0
11,163,316280,0,1,443,54.0,226.0,204.0,89.0,0,...,1,0,0,0,1,0,0,0,0,0
13,77,315922,0,1,443,45.0,61.0,0.0,234.0,0,...,1,0,0,1,0,0,0,0,0,0
15,204,233039,1,1,443,78.0,128.0,8.0,27.0,0,...,0,0,1,0,0,0,1,0,0,0
17,45,147535,0,1,80,142.0,250.0,178.0,1.0,0,...,1,0,0,0,0,0,0,0,0,0


In [55]:
scaler = StandardScaler()
y = data['label']
x = scaler.fit_transform(data.drop('label', axis=1))

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [57]:
x_train

array([[-1.20404238, -0.52078529,  0.23570226, ..., -0.16439899,
        -0.16439899,  0.        ],
       [ 1.2745573 ,  0.58358036,  0.23570226, ..., -0.16439899,
        -0.16439899,  0.        ],
       [-0.11307572, -0.24255515,  0.23570226, ..., -0.16439899,
        -0.16439899,  0.        ],
       ...,
       [-1.16576284, -0.70515851,  0.23570226, ..., -0.16439899,
        -0.16439899,  0.        ],
       [-0.14178537, -0.52137716,  0.23570226, ..., -0.16439899,
        -0.16439899,  0.        ],
       [-1.0222146 , -0.29051485,  0.23570226, ..., -0.16439899,
        -0.16439899,  0.        ]])

## Models

### Random Forest

In [58]:
## TODO might not need these starting bits

# Initialise the model
rfc = RandomForestClassifier()

# Train the model on the training data and the training data survivors
rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
# RF hyperparameters
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

Fitting 3 folds for each of 576 candidates, totalling 1728 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:   54.2s
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  1.3min


{'bootstrap': True, 'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}


[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed:  1.5min finished


In [60]:
# Make a new Random Forest model based on the best parameters found before
rfc = RandomForestClassifier(**grid_search.best_params_)

# Fit the model to the training data
rfc.fit(x_train, y_train)

# Get a predicition from the test data
pred = rfc.predict(x_test)

# Generate an accuracy from the prediction
rfc_acc = accuracy_score(y_test, pred)

print(rfc_acc)

0.5


In [61]:
rfc_confusion = confusion_matrix(y_test, pred)
print(rfc_confusion)

[[6 0]
 [6 0]]


In [62]:
rfc_classification = classification_report(y_test, pred, output_dict=True)
print(rfc_classification)

{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 6}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 12}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 12}}


C:\Users\huhon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Logistic Regression

In [63]:
# Initialise the model
lr = LogisticRegression()

# Fit the model with the training data
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter':[100, 500, 1000, 5000, 10000],
    'C': [0.01, 0.1, 1, 10, 100]
}

grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

Fitting 3 folds for each of 375 candidates, totalling 1125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 759 tasks      | elapsed:    0.6s


{'C': 10, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}


[Parallel(n_jobs=-1)]: Done 1125 out of 1125 | elapsed:    1.1s finished
C:\Users\huhon\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [65]:
# Make a new Logistic Regression model based on the best parameters found before
lr = LogisticRegression()#**grid_search.best_params_)

# Fit the model with the training data
lr.fit(x_train, y_train)

# Predict with the test data
pred = lr.predict(x_test)

# Get an accuracy score with the predicition
lr_acc = accuracy_score(y_test, pred)

print(lr_acc)

0.8333333333333334


In [66]:
lr_confusion = confusion_matrix(y_test, pred)
print(lr_confusion)

[[5 1]
 [1 5]]


In [67]:
lr_classification = classification_report(y_test, pred, output_dict=True)
print(lr_classification)

{'0': {'precision': 0.8333333333333334, 'recall': 0.8333333333333334, 'f1-score': 0.8333333333333334, 'support': 6}, '1': {'precision': 0.8333333333333334, 'recall': 0.8333333333333334, 'f1-score': 0.8333333333333334, 'support': 6}, 'accuracy': 0.8333333333333334, 'macro avg': {'precision': 0.8333333333333334, 'recall': 0.8333333333333334, 'f1-score': 0.8333333333333334, 'support': 12}, 'weighted avg': {'precision': 0.8333333333333334, 'recall': 0.8333333333333334, 'f1-score': 0.8333333333333334, 'support': 12}}


### K Nearest Neighbours

In [68]:
# Initialise model
knn = KNeighborsClassifier()

# Fit the model with the training data
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [69]:
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric':['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
    'leaf_size': [15, 30, 45, 60]
}

grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

Fitting 3 folds for each of 160 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s


{'leaf_size': 15, 'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}


[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:    0.3s finished


In [70]:
# Make a new KNN model based on the best parameters found before
knn = KNeighborsClassifier(**grid_search.best_params_)

# Fit the model to the scaled data
knn.fit(x_train, y_train)

# Get a prediction from the scaled test data
pred = knn.predict(x_test)

# Generate an accuracy score
knn_acc = accuracy_score(y_test, pred)

print(knn_acc)

0.8333333333333334


In [71]:
knn_confusion = confusion_matrix(y_test, pred)
print(knn_confusion)

[[6 0]
 [2 4]]


In [72]:
knn_classification = classification_report(y_test, pred, output_dict=True)
print(knn_classification)

{'0': {'precision': 0.75, 'recall': 1.0, 'f1-score': 0.8571428571428571, 'support': 6}, '1': {'precision': 1.0, 'recall': 0.6666666666666666, 'f1-score': 0.8, 'support': 6}, 'accuracy': 0.8333333333333334, 'macro avg': {'precision': 0.875, 'recall': 0.8333333333333333, 'f1-score': 0.8285714285714285, 'support': 12}, 'weighted avg': {'precision': 0.875, 'recall': 0.8333333333333334, 'f1-score': 0.8285714285714286, 'support': 12}}


### Support Vector Classifier

In [73]:
# Initialise model
svc = SVC()

# Fit the model to the training data
svc.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [74]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100, 1000],  
    'gamma': [2, 1, 0.1, 0.01, 0.001, 0.0001], 
    'kernel': ['rbf', 'linear', 'sigmoid']
}

grid_search = GridSearchCV(estimator = SVC(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s


{'C': 0.1, 'gamma': 2, 'kernel': 'linear'}


[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:    0.2s finished


In [75]:
# Make a new SVC model based on the best parameters found before
svc = SVC(**grid_search.best_params_)

# Fit model to scaled data
svc.fit(x_train, y_train)

# Predict using the scaled testing data
pred = svc.predict(x_test)

# Generate an accuracy score
svc_acc = accuracy_score(y_test, pred)

print(svc_acc)

0.9166666666666666


In [76]:
svc_confusion = confusion_matrix(y_test, pred)
print(svc_confusion)

[[5 1]
 [0 6]]


In [77]:
svc_classification = classification_report(y_test, pred, output_dict=True)
print(svc_classification)

{'0': {'precision': 1.0, 'recall': 0.8333333333333334, 'f1-score': 0.9090909090909091, 'support': 6}, '1': {'precision': 0.8571428571428571, 'recall': 1.0, 'f1-score': 0.923076923076923, 'support': 6}, 'accuracy': 0.9166666666666666, 'macro avg': {'precision': 0.9285714285714286, 'recall': 0.9166666666666667, 'f1-score': 0.916083916083916, 'support': 12}, 'weighted avg': {'precision': 0.9285714285714285, 'recall': 0.9166666666666666, 'f1-score': 0.9160839160839161, 'support': 12}}


### Evaluation

In [79]:
models = pd.DataFrame({
    'Model': ['Random Forest','Logistic Regression','K Nearest Neighbours','Support Vector Classifier'],
    'Score': [rfc_acc,lr_acc,knn_acc,svc_acc ]})

models.sort_values(by='Score', ascending=False)

,Model,Score
3,Support Vector Classifier,0.916667
1,Logistic Regression,0.833333
2,K Nearest Neighbours,0.833333
0,Random Forest,0.500000


In [2]:
matrices = [rfc_confusion, lr_confusion, knn_confusion, svc_confusion]

for i in [0, 1]:
    for matrix, label in zip(matrices, labels):
        print(f'{label}')
        print(matrix[i])
    print()

NameError: name 'rfc_confusion' is not defined